<a href="https://colab.research.google.com/github/oliverrmaa/sql-springboard/blob/main/solutions/SQL_solutions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Connecting Jupyter / Colab Notebook with Google BigQuery 

## 1.1 Discussion: Why do we need to learn SQL?

## 1.2 Google BigQuery Set Up 

1. Go to https://cloud.google.com/bigquery/docs/sandbox and click on “Go to the Cloud Console” 

2. Sign in with a google account, if you do not have one you can make one by making a gmail account very easily here https://www.google.com/gmail/about/

3. Go to the left hand panel and scroll down until you see BigQuery which is under the "Big Data" headings

4. Click on BigQuery (Protip: click on the pin button beside BigQuery so that it is the first thing that appears in the left hand menu)

5. BigQuery will now open and you will be prompted to create a new project. You can name this project anything you want. 

6. Once this is created make sure to bookmark the web URL to quickly get here in the future.



### 1.2.1 Google SDK Installation (only necessary for local notebooks)

If you prefer to use a local instance of Jupyter Notebook, please note you will have to install the Google Cloud SDK so that you have access to the `gcloud` package commands for authentication purposes. Please follow the instructions here: https://cloud.google.com/sdk/docs/install

### 1.2.2 Google BigQuery UI Exploration

This will be done in the BQ UI during the lesson.

## 1.2.3 Google BigQuery Public Datasets 

In the search bar of the left panel in Google BigQuery, please type in `public-data`. This should locate the `bigquery-public-data` project. Make sure to pin this project and you will have access to large publically available curated datasets! We will be using the `bigquery-public-data.chicago_crime.crime` table.

## 1.4 Notebook-BQ Connection/Authentication 

In [ ]:
# Use gcloud auth for authentication if you are using a local Jupyter Notebook 
# ! gcloud auth login
# ! pip install pandas
# Depending on the version of pandas, the following depedency may also be needed
# ! pip install pandas-gbq
from google.colab import auth
auth.authenticate_user()
import pandas as pd 

In [ ]:
%%time

query = """
  SELECT 
    unique_key,
    case_number,
    primary_type,
    date,
    longitude,
    latitude
  FROM `bigquery-public-data.chicago_crime.crime`
  WHERE year = 2020
"""

df = pd.read_gbq(query, project_id='sandbox-264306', dialect='standard')

CPU times: user 3.38 s, sys: 46.6 ms, total: 3.43 s
Wall time: 12.4 s


In [ ]:
%%time

%%bigquery df --project sandbox-264306 

SELECT 
  unique_key,
  case_number,
  primary_type,
  date,
  longitude,
  latitude
FROM `bigquery-public-data.chicago_crime.crime`
WHERE year = 2020

CPU times: user 3.76 s, sys: 63 ms, total: 3.82 s
Wall time: 17.4 s


## Interview Questions

### 1)	State the differences between HAVING and WHERE clauses

In [ ]:
%%time

%%bigquery df --project sandbox-264306 

WITH aggregation AS (

  SELECT 
    primary_type,
    COUNT(DISTINCT case_number) AS count_case_number
  FROM `bigquery-public-data.chicago_crime.crime`
  WHERE year = 2021
  GROUP BY primary_type

)

SELECT *
FROM aggregation
WHERE count_case_number > 100
ORDER BY primary_type

CPU times: user 77.1 ms, sys: 26.2 ms, total: 103 ms
Wall time: 1.83 s


In [ ]:
%%time

%%bigquery df --project sandbox-264306 

SELECT 
  primary_type,
  COUNT(DISTINCT case_number) AS count_case_number
FROM `bigquery-public-data.chicago_crime.crime`
WHERE year = 2021
GROUP BY primary_type
HAVING count_case_number > 100
ORDER BY primary_type

CPU times: user 49.5 ms, sys: 5.91 ms, total: 55.4 ms
Wall time: 1.9 s


### 2)	What is a Primary Key and what is a Foreign key?

In [ ]:
%%time

%%bigquery df --project sandbox-264306 

# primary (unique) key
SELECT 
  IF(
    COUNT(*) = COUNT(DISTINCT unique_key),
    TRUE,
    FALSE
  ) AS check_unique
FROM `bigquery-public-data.chicago_crime.crime`

CPU times: user 134 ms, sys: 14.5 ms, total: 149 ms
Wall time: 6.11 s


### 3)	What is COALESCE function

In [ ]:
%%time

%%bigquery df --project sandbox-264306 

# returns the first non-null column
SELECT 
  COALESCE(location_description, block) AS filled_in_location_description
FROM `bigquery-public-data.chicago_crime.crime`
WHERE location_description IS NULL

CPU times: user 73.6 ms, sys: 1.72 ms, total: 75.3 ms
Wall time: 1.8 s


### 4)	Define different types of Joins

In [ ]:
%%time

%%bigquery df --project sandbox-264306  

# Tables to demonstrate LEFT, RIGHT, INNER, FULL
WITH _2020 AS (

  SELECT 
    primary_type,
    COUNT(DISTINCT case_number) AS count_cases
  FROM `bigquery-public-data.chicago_crime.crime`
  WHERE year = 2020
  GROUP BY primary_type 

),

_2021 AS (

  SELECT 
    primary_type,
    COUNT(DISTINCT case_number) AS count_cases
  FROM `bigquery-public-data.chicago_crime.crime`
  WHERE year = 2021
  GROUP BY primary_type 

)

SELECT 
  a.primary_type,
  a.count_cases AS _2020_cases,
  b.count_cases AS _2021_cases
FROM _2021 AS a
LEFT JOIN _2020 AS b
ON a.primary_type = b.primary_type


CPU times: user 54.6 ms, sys: 6.35 ms, total: 60.9 ms
Wall time: 2.35 s


In [ ]:
%%time

%%bigquery df --project sandbox-264306  

# cross join
WITH _2021 AS (

  SELECT DISTINCT 
    case_number,
    longitude,
    latitude
  FROM `bigquery-public-data.chicago_crime.crime`
  WHERE year = 2021
  AND DATE(date) BETWEEN "2021-05-01" AND "2021-06-01"

),

cartesian_product AS (

  SELECT 
    a.case_number,
    b.case_number AS nearest_case,
    ST_DISTANCE(ST_GEOGPOINT(a.longitude, a.latitude), ST_GEOGPOINT(b.longitude, b.latitude)) AS distance,
    ROW_NUMBER() OVER w AS rn
  FROM _2021 AS a
  CROSS JOIN _2021 AS b
  WHERE 
    ST_DISTANCE(ST_GEOGPOINT(a.longitude, a.latitude), ST_GEOGPOINT(b.longitude, b.latitude)) IS NOT NULL
  WINDOW w AS (
    PARTITION BY a.case_number 
    ORDER BY ST_DISTANCE(
      ST_GEOGPOINT(a.longitude, a.latitude), 
      ST_GEOGPOINT(b.longitude, b.latitude)
    ) ASC
  )

)

SELECT *
FROM cartesian_product 
WHERE rn = 2

CPU times: user 299 ms, sys: 21.6 ms, total: 321 ms
Wall time: 7 s


### 5)	Give a scenario where self-join can be useful

Both the nearest distance query and the comparison of primary types across years joins.

### 6) What is DATE_ADD in SQL

In [ ]:
%%time

%%bigquery df --project sandbox-264306  

SELECT 
  unique_key,
  case_number,
  primary_type,
  date,
  longitude,
  latitude
FROM `bigquery-public-data.chicago_crime.crime`
WHERE year = EXTRACT(YEAR FROM CURRENT_DATE("America/Los_Angeles"))
AND DATE(date, "America/Los_Angeles") >= DATE_ADD(CURRENT_DATE("America/Los_Angeles"), INTERVAL -21 DAY)

### 7)	What is the difference between a view and a table

view is a "virtual table", it is the result set of an underlying query.

In [ ]:
%%time

%%bigquery df --project sandbox-264306  

CREATE OR REPLACE VIEW `sandbox-264306.sample_tables.example_view` AS (

  WITH _2020 AS (

    SELECT 
      primary_type,
      COUNT(DISTINCT case_number) AS count_cases
    FROM `bigquery-public-data.chicago_crime.crime`
    WHERE year = 2020
    GROUP BY primary_type 

  ),

  _2021 AS (

    SELECT 
      primary_type,
      COUNT(DISTINCT case_number) AS count_cases
    FROM `bigquery-public-data.chicago_crime.crime`
    WHERE year = 2021
    GROUP BY primary_type 

  )

  SELECT 
    a.primary_type,
    a.count_cases AS _2020_cases,
    b.count_cases AS _2021_cases
  FROM _2021 AS a
  LEFT JOIN _2020 AS b
  ON a.primary_type = b.primary_type

)

CPU times: user 32.6 ms, sys: 3.06 ms, total: 35.7 ms
Wall time: 1.23 s


### 8)	What is a Subquery? Explain a situation where sub query is used

"Contriversal" answer is that technically there are rarely situations where sub-query would be more beneficial than a common-table expression in most database systems. It's basically a query within a query. Subqueries can be used with `SELECT`, `FROM`, `WHERE`, or `HAVING`.

In [ ]:
%%time

%%bigquery df --project sandbox-264306 

SELECT DISTINCT 
  case_number,
  longitude,
  latitude
FROM `bigquery-public-data.chicago_crime.crime`
WHERE year = (SELECT MAX(year) FROM `bigquery-public-data.chicago_crime.crime`)

CPU times: user 949 ms, sys: 69.9 ms, total: 1.02 s
Wall time: 6.82 s


### 9) What is the difference between BETWEEN and IN operators in SQL?

In [ ]:
%%time

%%bigquery df --project sandbox-264306 

SELECT DISTINCT 
  case_number,
  longitude,
  latitude
FROM `bigquery-public-data.chicago_crime.crime`
WHERE year BETWEEN 2015 AND 2018

CPU times: user 11.1 s, sys: 146 ms, total: 11.3 s
Wall time: 47.5 s


In [ ]:
%%time

%%bigquery df --project sandbox-264306 

SELECT DISTINCT 
  case_number,
  longitude,
  latitude
FROM `bigquery-public-data.chicago_crime.crime`
WHERE year IN (2015, 2018)

CPU times: user 5.74 s, sys: 73.8 ms, total: 5.81 s
Wall time: 20.8 s


### 10)	What are the Aggregate functions are available in SQL?

In [ ]:
%%time

%%bigquery df --project sandbox-264306 

WITH year_block_agg AS (

  SELECT
    year,
    block,
    COUNT(case_number) AS block_case_count
  FROM `bigquery-public-data.chicago_crime.crime`
  GROUP BY year, block

)

SELECT 
  year,
  MAX(block_case_count) AS max_block_case_count,
  MIN(block_case_count) AS min_block_case_count,
  AVG(block_case_count) AS avg_block_case_count,
  SUM(block_case_count) AS sum_case_count
FROM year_block_agg
GROUP BY year
ORDER BY year

CPU times: user 138 ms, sys: 45.6 ms, total: 184 ms
Wall time: 4.69 s


### 11) What is the second highest value? (Window Function)

In [ ]:
%%time 

%%bigquery df --project sandbox-264306

WITH daily_agg AS (

  SELECT
    DATE(date) AS date,
    year,
    COUNT(DISTINCT case_number) AS daily_count
  FROM `bigquery-public-data.chicago_crime.crime`
  WHERE year IN (2019, 2020)
  GROUP BY date, year

),

row_numbered AS (

  SELECT 
    *,
    ROW_NUMBER() OVER (PARTITION BY year ORDER BY daily_count DESC) AS rn
  FROM daily_agg

)

SELECT * 
FROM row_numbered 
WHERE rn = 2
